In [11]:
# Imports
import pandas as pd

In [17]:
# Merging clients and dispositions
clients = pd.read_csv("dataset/client.csv", sep=";")

disps = pd.read_csv("dataset/disp.csv", sep=";")

clients_disps = pd.merge(clients, disps, on="client_id")

5369
5369
5369


In [49]:
# Merging clients/dispositions and cards
cards = pd.read_csv("dataset/card_train.csv", sep=";")

clients_disps.rename(columns={"type": "type_disp"}, inplace=True)

cards.rename(columns={"type": "type_card"}, inplace=True)

clients_disps_cards = pd.merge(clients_disps, cards, on="disp_id", how="outer")

,client_id,birth_number,district_id,disp_id,account_id,type_disp,card_id,type_card,issued
0,1,706213,18,1,1,OWNER,NaN,NaN,NaN
1,2,450204,1,2,2,OWNER,NaN,NaN,NaN
2,3,406009,1,3,2,DISPONENT,NaN,NaN,NaN
3,4,561201,5,4,3,OWNER,NaN,NaN,NaN
4,5,605703,5,5,3,DISPONENT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
5364,13955,456030,1,13647,11349,OWNER,NaN,NaN,NaN
5365,13956,430406,1,13648,11349,DISPONENT,NaN,NaN,NaN
5366,13968,680413,61,13660,11359,OWNER,1247.0,classic,950613.0
5367,13971,626019,67,13663,11362,OWNER,NaN,NaN,NaN


In [55]:
# Cleaning district data
districts = pd.read_csv("dataset/district.csv", sep=";")
# Renaming code to district_id
districts.rename(columns={"code ": "district_id"}, inplace=True)
# Removing name and region
districts.drop(['name ','region'], axis=1, inplace=True)

In [59]:
# Merging districts
clients_disps_cards_districts = pd.merge(clients_disps_cards, districts, on="district_id")

clients_disps_cards_districts.drop(['district_id'], inplace=True, axis=1)

In [62]:
# Merging accounts
accounts = pd.read_csv("dataset/account.csv", sep=";")
clients_disps_cards_districts_accounts = pd.merge(clients_disps_cards_districts, accounts, on="account_id")
clients_disps_cards_districts_accounts

,client_id,birth_number,disp_id,account_id,type_disp,card_id,type_card,issued,no. of inhabitants,no. of municipalities with inhabitants < 499,...,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,district_id,frequency,date
0,1,706213,1,1,OWNER,NaN,NaN,NaN,70699,60,...,65.3,8968,2.83,3.35,131,1740,1910,18,monthly issuance,950324
1,420,780313,420,343,OWNER,NaN,NaN,NaN,70699,60,...,65.3,8968,2.83,3.35,131,1740,1910,18,monthly issuance,940426
2,499,355708,499,413,OWNER,NaN,NaN,NaN,70699,60,...,65.3,8968,2.83,3.35,131,1740,1910,18,monthly issuance,960709
3,519,800413,519,431,OWNER,NaN,NaN,NaN,70699,60,...,65.3,8968,2.83,3.35,131,1740,1910,18,monthly issuance,961105
4,682,791021,682,568,OWNER,NaN,NaN,NaN,70699,60,...,65.3,8968,2.83,3.35,131,1740,1910,18,monthly issuance,950328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5364,9930,720623,9622,8039,OWNER,NaN,NaN,NaN,81344,61,...,55.3,8899,3.39,3.97,149,2987,2487,9,monthly issuance,951217
5365,10070,740423,9762,8153,OWNER,NaN,NaN,NaN,81344,61,...,55.3,8899,3.39,3.97,149,2987,2487,9,monthly issuance,970702
5366,11266,380925,10958,9153,OWNER,NaN,NaN,NaN,81344,61,...,55.3,8899,3.39,3.97,149,2987,2487,9,monthly issuance,961212
5367,11267,365826,10959,9153,DISPONENT,NaN,NaN,NaN,81344,61,...,55.3,8899,3.39,3.97,149,2987,2487,9,monthly issuance,961212


In [93]:
# Normalizing dates and adding sex
print(clients_disps_cards_districts_accounts)
dates = clients_disps_cards_districts_accounts["birth_number"]
dates = [str(date)[:2] + "-" + str(date)[2:4] + "-" + str(date)[4:] for date in dates]

new_dates = []
sex = []
for date in dates:
    month = int(date[3:5])
    gender = ""
    if month > 12:
        month = month - 50
        gender = "female"
    else:
        gender = "male"
    if month < 10:
        month = "0" + str(month)
    else:
        month = str(month)
    new_dates.append(date[0:2] + "-" + month + "-" + date[6:8])
    sex.append(gender)
    
ages = []
for date in new_dates:
    year = int("19" + date[0:2])
    age = 2021 - year
    ages.append(age)
    
#date_sex = {"age": ages, "sex": sex}
#date_sex_df = pd.DataFrame(date_sex)

clients_disps_cards_districts_accounts.drop(["birth_number"], inplace=True, axis=1)

people_account_data = clients_disps_cards_districts_accounts.assign(sex = sex, age = ages)
people_account_data

      client_id  disp_id  account_id  type_disp  card_id type_card  issued  \
0             1        1           1      OWNER      NaN       NaN     NaN   
1           420      420         343      OWNER      NaN       NaN     NaN   
2           499      499         413      OWNER      NaN       NaN     NaN   
3           519      519         431      OWNER      NaN       NaN     NaN   
4           682      682         568      OWNER      NaN       NaN     NaN   
...         ...      ...         ...        ...      ...       ...     ...   
5364       9930     9622        8039      OWNER      NaN       NaN     NaN   
5365      10070     9762        8153      OWNER      NaN       NaN     NaN   
5366      11266    10958        9153      OWNER      NaN       NaN     NaN   
5367      11267    10959        9153  DISPONENT      NaN       NaN     NaN   
5368      11701    11393        9504      OWNER      NaN       NaN     NaN   

      no. of inhabitants  no. of municipalities with inhabitant

KeyError: 'birth_number'